In [79]:
from sqlalchemy.sql import text
import sqlalchemy as sa
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import datetime

engine = create_engine('mysql+pymysql://root@localhost:8081/airport-analytics?charset=utf8mb4')

# FOREIGN KEY tailnum_fk(tailnum) REFERENCES planes (tailnum), YES
# FOREIGN KEY dest_fk(dest) REFERENCES airports (faa), YES

In [80]:
airports = pd.read_csv('../static/airports.csv')
airports.fillna(np.nan, inplace = True)
airports = airports[airports['faa'].str.match(pat = "^[A-Z0-9]{3}$", na=False) == True]

weather = pd.read_csv('../static/weather.csv')
weather.fillna(np.nan, inplace = True)
weather.replace(r'\s+', np.nan, regex=True, inplace=True)
weather['time_hour'] = pd.to_datetime(weather['time_hour'], errors='coerce').values.astype('datetime64[us]')

airlines = pd.read_csv('../static/airlines.csv')
airlines.fillna(np.nan, inplace = True)

planes = pd.read_csv('../static/planes.csv')
planes.fillna(np.nan, inplace = True)
planes['speed'].replace(r'\s+', np.nan, regex=True, inplace=True)
planes['year'].replace(r'\s+', np.nan, regex=True, inplace=True)

In [3]:
airports.to_sql('airports', con=engine, if_exists='append', index = False)
weather.to_sql('weather', con=engine, if_exists='append', index = False)
airlines.to_sql('airlines', con=engine, if_exists='append', index = False)
planes.to_sql('planes', con=engine, if_exists='append', index = False)

In [89]:
flights = pd.read_csv('../static/flights.csv')
flights.fillna(np.nan, inplace = True)
flights['time_hour'] = pd.to_datetime(flights['time_hour'], errors='coerce').values.astype('datetime64[us]')
flights['arr_delay'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['air_time'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['arr_time'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['dep_time'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['dep_delay'].replace(r'\s+', np.nan, regex=True, inplace=True)

In [96]:
baseCsvRowCount = len(flights)

flightsTotalClean = flights[
    flights.dest.isin(airports['faa']) & 
    flights.tailnum.isin(planes['tailnum']) &
    flights.origin.isin(weather['origin']) &
    flights.carrier.isin(airlines['carrier']) &
    flights.origin.isin(airports['faa']) &
    flights.year.isin(weather['year']) &
    flights.month.isin(weather['month']) &
    flights.day.isin(weather['day']) &
    flights.hour.isin(weather['hour'])
    ]

print('Lignes par défaut : ', len(flights))
print('Nettoyage complet : ', len(flightsTotalClean), '\n')

flightsAirportsFfaFk = flights[
    flights.dest.isin(airports['faa'])
]
flightsPlanesTailnumFk = flights[
     flights.tailnum.isin(planes['tailnum'])
]

flightsAirportsFfaFkLength = len(flightsAirportsFfaFk)
flightsPlanesTailnumFkLength = len(flightsPlanesTailnumFk)

print('fk airports_ffa : ', baseCsvRowCounts - flightsAirportsFfaFkLength)
print('fk planes_tailnum: ', baseCsvRowCounts - flightsPlanesTailnumFkLength)

flightsAirportFfaAndPlanesTailnumFk = flights[
    flights.dest.isin(airports['faa']) &
    flights.tailnum.isin(planes['tailnum'])
]

totalRemovedRows = (baseCsvRowCounts - len(flightsAirportFfaAndPlanesTailnumFk))
    
print('\nTotal des lignes supprimés : ', totalRemovedRows)

print('Supposé lignes de fin : ', baseCsvRowCounts - totalRemovedRows)

Lignes par défaut :  336776
Nettoyage complet :  277977 

fk airports_ffa :  7602
fk planes_tailnum:  52606

Total des lignes supprimés :  58799
Supposé lignes de fin :  277977


In [ ]:
flights.to_sql('flights', con=engine, if_exists='append', index = False)

In [ ]:
for i in range(len(flights.index)):
    try:
        row = flights.iloc[[i]]

        row.to_sql('flights', con=engine, if_exists='append', index = False)

    except Exception as e:
        continue